In [259]:
import pandas as pd
import numpy as np

In [260]:
df = pd.read_csv('TD_HOSPITAL_TRAIN.csv')

In [261]:
df.drop(['dose','pdeath'],axis=1)
for index in range(len(df['sex'])):
    val = df['sex'][index]
    if val == "male" or val == "Male" or val == "M" or val == 1:
        df['sex'][index] = 0
    else:
        df['sex'][index] = 1

# print(df['sex'])

def attempt_convert_to_float(column):
    try:
        return column.astype(float)
    except ValueError:  # If conversion fails, return the original column
        return column

# Apply the function to each column
df = df.apply(attempt_convert_to_float)

for col in df.columns:
    if df.dtypes[col] == "float64":
        df[col].fillna(df[col].mean(), inplace=True)

# df = pd.get_dummies(df, columns=['sex', 'race', 'primary'], drop_first=True)
df = pd.get_dummies(df, columns=['dnr','race', 'primary', 'disability', 'income' ,'extraprimary', 'cancer' ], drop_first=True)

df.to_csv('output.csv')

/var/folders/6w/qt75n21508bdvzq5z1lgl6sr0000gn/T/ipykernel_8802/3518595214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'][index] = 0
/var/folders/6w/qt75n21508bdvzq5z1lgl6sr0000gn/T/ipykernel_8802/3518595214.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'][index] = 1


In [262]:

correlation = df.corr()['death'].sort_values(ascending=False)
# print(correlation)
# threshold = float(input("Enter the threshold value you would like to analyze:"))
# print(threshold)

threshold = 0.1

# Identify features that have a correlation magnitude above the threshold
significant_features = correlation[correlation.abs() > threshold].index.tolist()

# Exclude the target variable 'death' from the features list
significant_features.remove('death')

new_df = df[significant_features + ['death']]

new_df.to_csv('sig.csv')


In [263]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

df = pd.read_csv('sig.csv')

# Split data into features and target variable
X = df.drop('death', axis=1)
y = df['death']

# Standardize numerical features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

regulizer = tf.keras.regularizers.l2(0.001)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(200, activation='relu', kernel_regularizer=regulizer),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])


history = model.fit(X_train, y_train, 
                    epochs=100, 
                    batch_size=60, 
                    validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Sample prediction
sample_data = X_test.iloc[0]  # get the first row from the test set
predicted_prob = model.predict(np.array([sample_data]))[0][0]
# print(f"The probability of death is: {predicted_prob * 100:.2f}%")
# predicted_label = (predicted_prob > 0.5).astype(int)
# print(f"Predicted Label: {predicted_label}")



Epoch 1/100
83/83 [==============================] - 0s 2ms/step - loss: 0.3699 - accuracy: 0.8421 - val_loss: 0.3711 - val_accuracy: 0.8447
Epoch 2/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3387 - accuracy: 0.8571 - val_loss: 0.3566 - val_accuracy: 0.8466
Epoch 3/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3325 - accuracy: 0.8597 - val_loss: 0.3522 - val_accuracy: 0.8470
Epoch 4/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3319 - accuracy: 0.8585 - val_loss: 0.3535 - val_accuracy: 0.8480
Epoch 5/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3263 - accuracy: 0.8587 - val_loss: 0.3481 - val_accuracy: 0.8475
Epoch 6/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3234 - accuracy: 0.8621 - val_loss: 0.3413 - val_accuracy: 0.8513
Epoch 7/100
83/83 [==============================] - 0s 1ms/step - loss: 0.3215 - accuracy: 0.8646 - val_loss: 0.3543 - val_accuracy: 0.8517
Epoch 8/100
8